In [1]:
# Instalação das dependências
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
from groq import Groq

# Carregamento das variaveis de ambiente (arquivo .env)
load_dotenv()

True

In [3]:
# Cria um cliente da API Groq usando a chave 
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Prompt de test para garantir que o modelo está funcionando
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-70b-8192",
    temperature=0
)

# Imprime a resposta do modelo
print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in several aspects:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in customer-facing applications where delayed responses can lead to frustration and a negative user experience.
2. **Low-Latency Inference**: Fast language models can perform inference (i.e., make predictions or generate text) rapidly, which is essential for applications that require immediate responses, such as:
	* Sentiment analysis for social media monitoring
	* Text classification for spam detection
	* Language translation for real-time communication
3. **Scalability**: Fast language models can handle large volumes of data and scale to meet the demands of large-scale applications, such as:
	* Processing massive amounts of tex

In [4]:
# Classe que define um agente inteligente
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []

        # Prompt inicial para configuração do modelo
        if self.system:
            self.messages.append({"role": "system", "content": system})

    # Função que faz o overload do operador (), isso permite que a classe seja chamada como se fosse uma função
    # Essa função recebe um prompt, envia esse prompt para o modelo, juntamente com o contexto anterior,
    # e recebe a resposta, a resposta também é salva no contexto, o contexto com todas as mensagens está na variável messages
    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    # Envia messages para a api da Groq
    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [5]:
# Prompt inicial usado para configurar o modelo, esse prompt ensina o LLM sobre como ele deve responder
# de acordo com as etapas do loop, ou seja Question, Thought, Action e Pause.
# Ele também ensina sobre as actions, ou tools, que estão disponíveis

system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()


# Tool para performar operações matemáticas
def calculate(operation: str) -> float:
    return eval(operation)


# Tool para descobrir a massa dos planetas do sistema solar
def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [ ]:
import re # Regex

# Loop ReAct
def loop(max_iterations=10, query: str = ""):

    # Criação do agente
    agent = Agent(client=client, system=system_prompt)

    # Definição das tools, essas strings devem corresponder ao nome de uma função python disponível
    tools = ["calculate", "get_planet_mass"]

    next_prompt = query

    i = 0
  
    # Loop com um máximo de iterações para evitar que o agente pense para sempre
    while i < max_iterations:
        i += 1
        # Envia o prompt para o agente, aqui o agente é chamado como uma função, apesar de ser uma classe
        result = agent(next_prompt)
        print(result)


        if "PAUSE" in result and "Action" in result:
            # Se o LLM decidiu tomar uma ação, então execute a ação desejada
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            # O resultado da ação é enviado para o LLM na próxima iteração
            if chosen_tool in tools:
                # Chama uma das funções definidas em tools
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        # Se o LLM julga que uma resposta foi encontrada, então termina o processo
        if "Answer" in result:
            break


# Pergunta que inicia o processo
loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

Thought: I need to find the mass of Earth and Saturn, then add them together and multiply by 2.
Thought: I need to find the mass of Earth.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now I have the mass of Earth, I need to find the mass of Saturn.
Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: Now I have the mass of Earth and Saturn, I need to add them together.
Action: calculate: 5.972e+24 + 5.683e+26
PAUSE
Observation: 5.74272e+26
Thought: Now I have the sum of the masses, I need to multiply it by 2.
Action: calculate: 5.74272e+26 * 2
PAUSE
Observation: 1.148544e+27
Answer: The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e+27.
